# Objectives of the Notebook 

 *  **Building a neural network with the best accuracy possible and testing it's accuracy**
 
 *  **Give it a glance look at the data**
 
 *  **Showing the result in confusion matrix**
 
 

# Importing libraries

In [ ]:
import numpy as np 
import pandas as pd
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import itertools
from sklearn.metrics import f1_score

Inputing the data 

In [ ]:
df=pd.read_csv(r'../input/hotel-booking-demand/hotel_bookings.csv')

Is the data balanced ? To this we need to have all the data that is canceled and the data where is not canceled

Separating what is the canceled data.

In [ ]:
cancelled=df.loc[(df['is_canceled']==1)]

In [ ]:
a=100*(cancelled.shape[0]/df.shape[0])
print("Percentage of the canceled values  " + str(a))
print('Percentge of the non canceled values  '+ str(100-a))

As we can see the dataset is not balanced but the difference is not that big we can change our aproach by changing evaluation metrics.

In [ ]:
df.pop('reservation_status_date')
df.pop('arrival_date_year')

In [ ]:
df['hotel'].value_counts().sort_values().plot(kind='bar')
plt.title('Type of Hotel ')
plt.show()

In [ ]:
cancelled['hotel'].value_counts().sort_values().plot(kind='bar')
plt.title('type of hotel cancelled')
plt.show()

In [ ]:
df['hotel'].value_counts(normalize=True).sort_values().plot(kind='bar')
plt.title('Type of Hotel (normalized)')
plt.show()

In [ ]:
cancelled['hotel'].value_counts(normalize=True).sort_values().plot(kind='bar')
plt.title('Normalized type of hotel cancelled')
plt.show()

In [ ]:
df['arrival_date_month'].value_counts().sort_values().plot(kind = 'bar')
plt.title('Months of arrival (General)')
plt.show()

In [ ]:
cancelled['arrival_date_month'].value_counts().sort_values().plot(kind='bar')
plt.title('Date of supossed arrival of people that canceled')
plt.show()

In [ ]:
print(df['adults'].value_counts())

Removing outliers

In [ ]:
df=df.loc[(df['adults']<20)]

In [ ]:
df['adults'].value_counts().sort_values().plot(kind='bar')
plt.title('No of Adults (Outliers removed)')
plt.show()

In [ ]:
cancelled['adults'].value_counts().sort_values().plot(kind='bar')
plt.title('No of adults of people that cancelled')
plt.show()

In [ ]:
print(df['children'].value_counts())

Another outlier

In [ ]:
df=df.loc[(df['children']<10)]
df['children'].value_counts().sort_values().plot(kind='bar')
plt.title('General Children')
plt.show()

In [ ]:
plt.figure(figsize=(30,15))
df['country'].value_counts().sort_values().plot(kind='bar')
plt.show()
print(df['country'].value_counts())

In [ ]:
# How much of it with meal 
df['meal'].value_counts().sort_values().plot(kind='bar')
plt.title('General Meal')
plt.show()

In [ ]:
# How much of it with meal canceled
cancelled['meal'].value_counts().sort_values().plot(kind='bar')
plt.title('General Meal(Canceled)')
plt.show()

In [ ]:
df['deposit_type'].value_counts().sort_values().plot(kind='bar')
plt.title('deposit type')
plt.show()

In [ ]:
cancelled['deposit_type'].value_counts().sort_values().plot(kind='bar')
plt.title('Cancelled deposit type')
plt.show()

Transforming Categorical to Numerical

In [ ]:
for col_name in df.columns:
    if(df[col_name].dtype == 'object'):
        df[col_name]= df[col_name].astype('category')
        df[col_name] = df[col_name].cat.codes

Looking at the number of nan and replacing then by zero 

In [ ]:
df=df.fillna(0)
print (df.isna().any(axis=-0))

# Building our model using tensor flow

In [ ]:

y=np.array(df.pop('is_canceled'))

x=df.values

x_train,x_test,y_train,y_test=train_test_split(x,y)

In [ ]:
epochs=15

model = keras.Sequential([
    keras.layers.Dense(32,input_shape=(29,),activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history=model.fit(x_train, y_train, epochs=epochs,validation_split=0.1,verbose=0)


In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:

history=history.history
epochs_num=np.arange(1,epochs+1)

plt.figure(0)
plt.title('accuracy')
plt.plot(epochs_num,history['accuracy'],label='accuracy')
plt.plot(epochs_num,history['val_accuracy'],label='val_accuracy')
plt.legend()
plt.show()

plt.figure(0)
plt.title('loss')
plt.plot(epochs_num,history['loss'],label='loss')
plt.plot(epochs_num,history['val_loss'],label='accuracy_loss')
plt.legend()
plt.show()

comp=model.predict(x_test)
comp=np.array([np.argmax(u) for u in comp])
cm = confusion_matrix(y_true=y_test, y_pred=comp)
plot_confusion_matrix(cm=cm,classes=['Non Canceled','Canceled'],title='Confusion Matrix')
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
cm = confusion_matrix(y_true=y_test, y_pred=comp)
plot_confusion_matrix(cm=cm,classes=['Non Canceled','Canceled'],title='Confusion Matrix',normalize=True)
plt.show()
print('F1 score '+str(f1_score(y_true=y_test,y_pred=comp)))

# Conclusion

Being very sincere with the person who reads this notebook. I am very suspicious of the notebook results maybe there are hidden in some data something date relates directly when is canceled or not. 
If this is not true this result is amazing it predicted with a great precision showing that a neural network may be the best solution to the prediction of cancelation or not. 
Other possible solution is that i got lucky in the training of the neural net, but is really suspicious the precision being so high if someone have a clue what may be happening please leave a comment
   
   **If you enjoyed the Notebook give it an Up and Thank you**
